In [1]:
# 0. generate random order of edges
# 1. so that it actually starts with building a tree
# 2. build EPPDC for this tree
# 3. add visualizations
# now we iteratively add an edge (a,b)
# 4. add one direction as in the usual PPDC
# 5. bruteforce all sequences Q = b v1 v2 ... vt \inf
# 6. select the sequence which would give an EPPDC
# 7. add visualizations to the sequence process
# 8. if the process fails - log this example;
# 9.                        and try to bruteforce 4.
# 10. if the process fails again - this is the end of this approach

from collections import defaultdict
from copy import deepcopy
from enum import Enum
import random
import time

from unionfind import unionfind

In [2]:
# stuff

def gen_edges(n):
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            edges.append((i, j))
    random.shuffle(edges)
    return edges

def sort_edges_to_build_a_tree(edges, n, verbose=False):
    u = unionfind(n)
    tree_edges = []
    non_tree_edges = []
    for edge in edges:
        v1, v2 = edge
        if u.issame(v1, v2):
            non_tree_edges.append((v1, v2))
        else:
            u.unite(v1, v2)
            tree_edges.append(edge)
    random.shuffle(non_tree_edges)
    if verbose:
        print(f'{tree_edges=}')
    assert (len(tree_edges) == n - 1)
    return tree_edges, non_tree_edges


class Graph():
    def __init__(self, n, edges):
        self.n = n
        self.edges = edges
        self.neibs = dict()
        for v in range(n):
            self.neibs[v] = []
        for v1, v2 in edges:
            self.neibs[v1].append(v2)
            self.neibs[v2].append(v1)


class EdgePath():
    def __init__(self):
        self.edges = []
        self.vertices = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])

    def _reverse(self):
        self.edges.reverse()
        for edge_idx, edge in enumerate(self.edges):
            self.edges[edge_idx] = (edge[1], edge[0])
        self._start, self._finish = self._finish, self._start

    def _update_start(self, vertex):
        self._start = vertex
        self.ends = set([self._start, self._finish])

    def _update_finish(self, vertex):
        self._finish = vertex
        self.ends = set([self._start, self._finish])

    def append(self, edge):
        if self.edges:
            if self._start == edge[0]:
                self._reverse()
            assert self._finish == edge[0]
            assert edge[1] not in self.vertices
        self.edges.append(edge)
        for vertex in edge:
            self.vertices.add(vertex)
        if self._start is None:
            self._start = self.edges[0][0]
        self._update_finish(edge[-1])

    def shrink(self):
        assert self.edges is not None
        assert len(self.edges) > 0
        self.vertices.remove(self._finish)
        self.edges.pop()
        # FIXME: what if the lenght of path becomes 0? so far never happened
        assert len(self.edges) > 0
        self._update_finish(self.edges[-1][-1])

    def rebuild(self, new_vertices):
        self.edges = []
        self.vertices = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])
        for i in range(len(new_vertices) - 1):
            self.append((new_vertices[i], new_vertices[i + 1]))

    def __contains__(self, vertex):
        return vertex in self.vertices

    def __repr__(self):
        res = []
        for edge in self.edges:
            res.append(edge[0])
        res.append(self._finish)
        return str(res)


class EPPDC():
    def __init__(self, n):
        self.n = n
        self.paths = []

    def append(self, path):
        self.paths.append(path)

    def is_eppdc(self):
        u = unionfind(n)
        for path in self.paths:
            u.unite(*path.ends)
        return len(u.groups()) == 1

    def sanity_check(self):
        assert self.is_eppdc()
        end_counts = defaultdict(int)
        edge_counts = defaultdict(int)
        for path in self.paths:
            for end in path.ends:
                end_counts[end] += 1
            for edge in path.edges:
                sorted_edge = list(edge)
                sorted_edge.sort()
                sorted_edge = tuple(sorted_edge)
                edge_counts[sorted_edge] += 1
        for count in end_counts.values():
            assert (count == 2)
        for count in edge_counts.values():
            assert (count == 2)

    def calc_repeated_triples(self):
        triples = set()
        reps = 0
        for path in self.paths:
            edges = path.edges
            for edge_idx in range(len(edges) - 1):
                triple = (edges[edge_idx][0], edges[edge_idx][1], edges[edge_idx + 1][1])
                if triple in triples:
                    reps += 1
                triples.add(triple)
                triple = (edges[edge_idx + 1][1], edges[edge_idx][1], edges[edge_idx][0])
                triples.add(triple)
        return reps

    
    def calc_median_path_len(self):
        lens = []
        for path in self.paths:
            lens.append(len(path.edges))
        lens.sort()
#         lens.reverse()
        return tuple(lens)
#         return lens[len(lens)//2]

    # TODO:
    def calc_abs_sum_diffs_consec_path_lens(self):
        # find out which paths are consecutive to each other
        # calculate diffs of path lengths
        # return sum, or max diff (e. g., maybe idea is to always keep something similar to Knuth labeling
        # that consecutive paths have small diference in length between each other)
        res = 0  # sum
        for idx1, path1 in enumerate(self.paths):
            for idx2, path2 in enumerate(self.paths):
                if idx2 <= idx1:
                    continue
            ends1 = path1.ends
            ends2 = path2.ends
            if len(ends1 & ends2) == 0:
                continue
            res += abs(len(path1.edges) - len(path2.edges))
        return res


# DOESN'T WORK for K4
#     def has_unique_triples(self):
#         triples = set()
# #         print('debug')
#         for path in self.paths:
# #             print(f'{path=}')
#             edges = path.edges
#             for edge_idx in range(len(edges) - 1):
#                 triple = (edges[edge_idx][0], edges[edge_idx][1], edges[edge_idx + 1][1])
#                 if triple in triples:
# #                     print(f'{triple=}', 'vs', triples)
#                     return False
#                 triples.add(triple)
#                 triple = (edges[edge_idx + 1][1], edges[edge_idx][1], edges[edge_idx][0])
#                 if triple in triples:
# #                     print(f'{triple=}', 'vs', triples)
#                     return False
#                 triples.add(triple)
#         return True

# kind of DOESN'T WORK - we can have odd number of paths
#     def can_be_split_into_pairs(self):
#         return True

    def __iter__(self):
        return self.paths.__iter__()

    def __next__(self):
        return self.paths.__next__()


class DFSState():
    def __init__(self, n):
        self.visited = set()
        self.eppdc = EPPDC(n)
        self.cur_path = EdgePath()

    def dfs(self, v1, graph):
        self.visited.add(v1)

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.dfs(v2, graph)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        self.eppdc.append(self.cur_path)
        self.cur_path = EdgePath()

    def knuth(self, v1, graph, mode='bfs'):
        assert mode in ['bfs', 'dfs']
        next_mode = 'dfs'
        if mode == 'dfs':
            next_mode = 'bfs'

        self.visited.add(v1)

        if mode == 'bfs':
            if v1 != 0:
                self.eppdc.append(self.cur_path)
                self.cur_path = EdgePath()

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.knuth(v2, graph, next_mode)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        if mode == 'dfs' or v1 == 0:
            self.eppdc.append(self.cur_path)
            self.cur_path = EdgePath()


def build_tree_eppdc(graph, n):
    # idea: do some kind of dsf-like tree traversal: bfs, dfs, or Knuth
    # for simplicity we go with dfs
    # also: here we introduce the data structures, useful for describing EPPDC

    # algorithm: start at root, root will be 1 end of path
    # go 1 step deeper, add edge to path
    # go further, until the leaf
    # leaf will be a second end
    # start a new path at the leaf
    # continue in this manner
    # actual algorithm is: we start a new path each time we need to leave the vertex
    state = DFSState(n)
    state.dfs(0, graph)
#     state.knuth(0, graph, 'bfs')
    return state.eppdc

class State(Enum):
    SUCCESS = 0
    FAIL = 1
    BACKTRACK = 2

In [15]:
# THE ALGORITHM ITSELF
def extend(n, v1, v2, eppdc, part, use_backtrack=False, verbose=False):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2

    # 4. add one direction as in the usual PPDC
    # 5. bruteforce all sequences Q = b u1 u2 ... ut \inf
    # 6. select the sequence which would give an EPPDC
    iteration_count = 0
    do_backtrack = False
    original_v1 = v1
    while True:
        # Hacky ways to exit the unfortunate random choices
        iteration_count += 1

        if use_backtrack and iteration_count > n * 3:
            do_backtrack = True

        if part == 1:
            do_backtrack = False
        upper_bound = n ** 3

        if verbose:
            upper_bound = n ** 2

        if iteration_count >= upper_bound:
            return (State.FAIL, (v1, v2))

        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)
        random.shuffle(v1_paths)

        # FIXME:
#         if part == 1:
#             assert len(v1_paths) == 2
#         else:
#             # current state: 3 paths ending at v1, 1 path ending at v2
#             assert len(v1_paths) == 3

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
                if eppdc.is_eppdc():  # and eppdc.has_unique_triples() and eppdc.can_be_split_into_pairs():
                    return (State.SUCCESS, (v1, v2))
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but not eppdc')
            else:
                # example
                # v2 is in path
                # v1, v2 = (4, 3)
                # [2, 3, 1, 4]
                # =>
                # [2, 3, 4, 1]
                # v1, v2 = (1, 3)

                path_vertices = []
                for edge in path.edges:
                    path_vertices.append(edge[0])
                path_vertices.append(path._finish)
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    # FIXME: fix this code for part 1
                    if (v1 != original_v1) and do_backtrack:
                        assert (part == 2)
                        if part == 2:
                            if verbose:
                                print(f'  part{part}.x: try to backtrack')
                            new_path_vertices = path_vertices[:-1]
                            if verbose:
                                print(f'  before backtrack rebuild')
                                for debug_path in eppdc.paths:
                                    print(f'    {debug_path=}')
                            path.rebuild(new_path_vertices)
                            if not eppdc.is_eppdc():
                                if verbose:
                                    print('backtrack failed')
                                path.rebuild(path_vertices)
                                continue
                            else:
                                assert (eppdc.is_eppdc())
                                switched = True
                                if verbose:
                                    print(path_vertices, '=>', new_path_vertices)
                                    print(f'  after backtrack rebuild')
                                    for debug_path in eppdc.paths:
                                        print(f'    {debug_path=}')
                                return (State.BACKTRACK, (v1, v2))
                        else:
                            if verbose:
                                print('FIXME')
                            continue
                    else:
                        if v1 != original_v1:
                            if verbose:
                                print(f'  part{part}.x: could try to backtrack edge ({v1}, {v2})')
                        continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')
                assert (eppdc.is_eppdc())
                path.rebuild(new_path_vertices)
                if not (eppdc.is_eppdc()):  # and eppdc.has_unique_triples() and eppdc.can_be_split_into_pairs()):
                    if verbose:
                        print(f'  part{part}.x: not eppdc; tried {new_path_vertices[-1]}')
                    path.rebuild(path_vertices)
                    continue

                if verbose:
                    print('after rebuild')
                assert (eppdc.is_eppdc())
                switched = True
                if verbose:
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')
                v1 = new_path_vertices[-1]
                v1_paths = []
                for possible_v1_path in eppdc:
                    if v1 in possible_v1_path.ends:
                        v1_paths.append(possible_v1_path)
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break
#     return (State.FAIL, (v1, v2))  # FIXME, in case i remove has_unique_triples
    assert False

In [16]:
# DONE!

# check all the orderings
# n = 6
n = 7
# n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20

# verbose = True
verbose = False

print_stats = True
# print_stats = False

max_rep = 1000
max_rep = 100
# max_rep = 1

for rep0 in range(max_rep):
    if (rep0 % 10 == 0):
        print(rep0, '/', max_rep)
    seed = random.randint(0, 1000000000)
    
#     print(f'{seed=}')
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    
    random.seed(seed + rep0)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    tree_eppdc.sanity_check()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge

    # steps 4.-6.
    
    good_masks = 0
    all_masks = 0
#     assert ((n * (n - 1) // 2 - (n - 1)) == len(non_tree_edges))
    minimal_edge_idx_fail = None
    failed_spectacularly = False
    failed_starts = defaultdict(int)
    failed_spectacularly_starts = defaultdict(int)
    good_starts = defaultdict(int)

#     for mask_idx in range(50):
    for mask_idx in range(1):
#         mask = random.randint(0, 2 ** len(non_tree_edges) - 1)
#         mask = 0b11
        eppdc = deepcopy(tree_eppdc)

        all_masks += 1
        failed = False
        added_edges = []
        start = None
        for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
#             if edge_idx > 1:
#                 break
            eppdc_before_part1 = deepcopy(eppdc)

            next_eppdcs = []

            v1, v2 = orig_v1, orig_v2
#             if mask & (2 ** edge_idx) > 0:
            if random.random() > 0.5:
                v1, v2 = v2, v1
            added_edges.append((v1, v2))

            if edge_idx > 0:
                start = (added_edges[0], added_edges[1])

            if verbose:
                print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                for path in eppdc:
                    print(path)

#             reps = 20
            reps = 4
            for rep in range(reps):
                if rep > 0 and verbose:
                    print('')
                    print('try again')
#                 if rep > 0:
#                     v1, v2 = v2, v1
                eppdc = deepcopy(eppdc_before_part1)

                success, _ = extend(tree.n, v1, v2, eppdc, 1,
                                    use_backtrack=False, verbose=verbose)
                if verbose:
                    print(f'{edge_idx=} part1 success: {success}')
                    if success == State.SUCCESS:
                        print(f'{edge_idx=} after part 1; adding edge {(v2, v1)}')
                        for path in eppdc:
                            print(path)

                if success == State.SUCCESS:
                    success, _ = extend(tree.n, v2, v1, eppdc, 2,
                                        use_backtrack=False, verbose=verbose)

                if success != State.FAIL:
                    assert success == State.SUCCESS
                    if verbose:
                        print(f'after part 2')
                        for path in eppdc:
                            print(path)
                        print()
                        print()
                    eppdc.sanity_check()
                    break
                if verbose:
                    time.sleep(0.02)
            if success == State.FAIL:
                failed = True
                if minimal_edge_idx_fail is None or minimal_edge_idx_fail > edge_idx:
                    minimal_edge_idx_fail = edge_idx
                if edge_idx <= 1:
                    failed_spectacularly = True
                    start = (added_edges[0], added_edges[1])
                    failed_spectacularly_starts[start] += 1
                    print(f'{seed=}')
                    print(f'graph failed at {edge_idx}')
                    print('tree')
                    for path in tree_eppdc:
                        print(path)
                    print('added edges')
                    for edge in added_edges:
                        print(edge)
                    assert False
                break

        if not failed:
            good_masks += 1
            good_starts[start] += 1
        else:
            failed_starts[start] += 1
#     if failed_spectacularly or good_masks == 0 or print_stats:
    if good_masks/all_masks < 0.9:
        print(f'{seed=}')
        print(good_masks, all_masks, good_masks/all_masks, minimal_edge_idx_fail, len(non_tree_edges))
        print(f'{good_starts=}')
        print(f'{failed_starts=}')
        print(f'{failed_spectacularly_starts=}')
print('DONE')

0 / 100


KeyboardInterrupt: 

In [ ]:
# unfinished TODO:
# - let's try to add to a tree just 2 edges
# - find cases, where the addition of first edge in any direction
#   leads to 2 eppdc solutions
# - one of which is not extendable with some edge in some direction
#   and the other one is okay
# - and then check out, whether the other one is okay with adding any other edge in any other direction

In [ ]:
# unused TODO
# what if we keep several candidates for eppdc to continue

In [ ]:
# unfinished TODO: kind-of bruteforce all solutions
# idea: we keep the set of all non-failed solutions
# we test them on adding any of the remained edges
# but we proceed with 1 of the edges

# n = 7
# # n = 8
# # n = 6
# # n = 9
# # n = 10
# verbose = False

# for rep0 in range(100):
#     seed = random.randint(0, 10000000)
# #     seed = 3183013; n = 9
# #     seed = 3259030; n = 10
# #     seed = 7659167; n = 7
# #     seed = 2212498; n = 7
#     print(f'{seed=}')
#     random.seed(seed)

#     # 0. generate random order of edges
#     edges = gen_edges(n)
#     # 1. so that we actually start with building a tree
#     tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
#     random.seed(seed + rep0)

#     # 2. build EPPDC for this tree
#     tree = Graph(n=n, edges=tree_edges)
#     tree_eppdc = build_tree_eppdc(tree, n)
#     if verbose:
#         for path in tree_eppdc:
#             print(path)

#     # now we iteratively add an edge

#     # steps 4.-6.
#     eppdcs = [deepcopy(tree_eppdc)]

#     for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
#         if verbose:
#             print(f'before part 1; adding edge {(orig_v1, orig_v2)}')
#             for path in eppdc:
#                 print(path)

# #         eppdc_before_part1 = deepcopy(eppdc)

#         for solution_idx in range(len(graphs)):

#             for edge_idx2, (test_v1, test_v2) in enumerate(non_tree_edges[edge_idx:]):
#                 for orig_vs in [(test_v1, test_v2), (test_v2, test_v1)]:

#                     v1, v2 = orig_vs[0], orig_vs[1]

#                     reps = 50
#                     for rep in range(reps):
#                         graph = deepcopy(graph_before_part1)
#                         eppdc = deepcopy(eppdc_before_part1)

#                         success, _ = extend(tree.n, v1, v2, eppdc, 1,
#                                             use_backtrack=False, verbose=verbose)
#                         if success == State.SUCCESS:
#                             success, _ = extend(tree.n, v2, v1, eppdc, 2,
#                                                 use_backtrack=False, verbose=verbose)

#                         if success != State.FAIL:
#                             assert success == State.SUCCESS
#                             next_eppdcs.append(deepcopy(eppdc))
#                         if verbose:
#                             time.sleep(0.02)


#         next_eppdcs = []

#         for orig_vs in [(orig_v1, orig_v2), (orig_v2, orig_v1)]:
#             v1, v2 = orig_vs[0], orig_vs[1]

#             reps = 50
#             for rep in range(reps):
#                 eppdc = deepcopy(eppdc_before_part1)

#                 success, _ = extend(tree.n, v1, v2, eppdc, 1,
#                                     use_backtrack=False, verbose=verbose)
#                 if success == State.SUCCESS:
#                     success, _ = extend(tree.n, v2, v1, eppdc, 2,
#                                         use_backtrack=False, verbose=verbose)

#                 if success != State.FAIL:
#                     assert success == State.SUCCESS
#                     next_eppdcs.append(deepcopy(eppdc))
#                 if verbose:
#                     time.sleep(0.02)

#         if len(next_eppdcs) == 0:
#             print(f'fail on adding ({v1}, {v2})')
#             print(f'{edge_idx=}', 'vs', len(non_tree_edges))
#             for path in eppdc_before_part1:
#                 print(path)
#         elif edge_idx == 5:
#             print("didn't fail")
#             print(f'{edge_idx=}', 'vs', len(non_tree_edges))
#             for path in eppdc_before_part1:
#                 print(path)
#         assert (len(next_eppdcs) > 0)
#         best_solution_idx = None
#         best_val = None
#         best_val2 = None

# #         random.shuffle(next_eppdcs)

#         for solution_idx in range(len(next_eppdcs)):
#             cur_val = next_eppdcs[solution_idx].calc_abs_sum_diffs_consec_path_lens()
#             cur_val2 = next_eppdcs[solution_idx].calc_median_path_len()
# #             cur_val3 = next_eppdcs[solution_idx].calc_repeated_triples()

#             if best_val is None or (cur_val < best_val): # or ((cur_val == best_val) and (cur_val2 > best_val2)):
# #             if random.random() > 0.25:
#                 best_solution_idx = solution_idx
#                 best_val = cur_val
#                 best_val2 = cur_val2
#         eppdc = deepcopy(next_eppdcs[best_solution_idx])


In [124]:
# # check specific failing seeds
# # trying to add a different second edge
# # DONE: only 1 edge breaks; others are fine

# verbose = True
# verbose = False

# max_rep = 10

# for rep0 in range(max_rep):
#     if (rep0 % 100 == 0):
#         print(rep0, '/', max_rep)

#     # Fails on specific second edge with specific orientation
#     seed = 6798523; n = 6
#     seed = random.randint(0, 1000000000)
# #     seed = 1650216; n = 8
# #     seed = 997616; n = 7
# #     seed = 1631113; n = 9
# #     print(f'{seed=}')
#     random.seed(seed)

#     # 0. generate random order of edges
#     edges = gen_edges(n)
#     # 1. so that we actually start with building a tree
#     tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    
#     random.seed(seed + rep0)

#     # 2. build EPPDC for this tree
#     tree = Graph(n=n, edges=tree_edges)
#     tree_eppdc = build_tree_eppdc(tree, n)
#     tree_eppdc.sanity_check()
#     if verbose:
#         for path in tree_eppdc:
#             print(path)

#     # now we iteratively add an edge

#     # steps 4.-6.
    
#     good_masks = 0
#     all_masks = 0
# #     assert ((n * (n - 1) // 2 - (n - 1)) == len(non_tree_edges))
#     minimal_edge_idx_fail = None
#     failed_spectacularly = False
#     failed_starts = defaultdict(int)
#     failed_spectacularly_starts = defaultdict(int)
#     good_starts = defaultdict(int)

#     for mask_idx in range(100):
# #         mask = random.randint(0, 2 ** len(non_tree_edges) - 1)
#         graph = deepcopy(tree)
#         eppdc = deepcopy(tree_eppdc)

#         all_masks += 1
#         failed = False
#         added_edges = []
#         for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
# #             if edge_idx > 1:
# #                 break

#             if edge_idx == 1:
#                 graph_after_1_edge = deepcopy(graph)
#                 eppdc_after_1_edge = deepcopy(eppdc)

#             if edge_idx > 1:
#                 graph = deepcopy(graph_after_1_edge)
#                 eppdc = deepcopy(eppdc_after_1_edge)

#             graph_before_part1 = deepcopy(graph)
#             eppdc_before_part1 = deepcopy(eppdc)

#             next_graphs = []
#             next_eppdcs = []

#             v1, v2 = orig_v1, orig_v2
# #             if mask & (2 ** edge_idx) > 0:
#             if random.random() > 0.5:
#                 v1, v2 = v2, v1
#             if edge_idx <= 1:
#                 added_edges.append((v1, v2))
#             else:
#                 added_edges[1] = (v1, v2)

#             if verbose:
#                 print(f'before part 1; adding edge {(v1, v2)}')
#                 for path in eppdc:
#                     print(path)

#             reps = 20
#             for rep in range(reps):
#                 graph = deepcopy(graph_before_part1)
#                 eppdc = deepcopy(eppdc_before_part1)

#                 success, _ = extend(graph, v1, v2, eppdc, 1,
#                                     use_backtrack=False, verbose=verbose)
#                 if success == State.SUCCESS:
#                     success, _ = extend(graph, v2, v1, eppdc, 2,
#                                         use_backtrack=False, verbose=verbose)

#                 if success != State.FAIL:
#                     assert success == State.SUCCESS
#                     break
#                 if verbose:
#                     time.sleep(0.02)
#             if success == State.FAIL:
#                 failed_spectacularly = True
#                 start = (added_edges[0], added_edges[1])
#                 if verbose:
#                     print(f'{seed=}')
#                     print(f'graph failed at {edge_idx}')
#                     print('tree')
#                     for path in tree_eppdc:
#                         print(path)
#                     print('added edges')
#                     for edge in added_edges:
#                         print(edge)
#                 start = (added_edges[0], added_edges[1])
#                 failed_starts[start] += 1
#     if failed_spectacularly:
#         print(f'{seed=}')
#         print(f'{failed_starts=}')
#         print(non_tree_edges)
# print('DONE')

In [7]:
# n = 7
# seed=36303
# 27740 32768 0.8465576171875
# seed=1067458
# 915 1000 0.915


In [141]:
# TODO: best edge adding order?
# TODO: best spanning tree to start?

# TODO:
# seed = 3183013; n = 9
# fails on 4th edge, very interesting

n = 7
# n = 8
# n = 6
# n = 9
# n = 10
verbose = False

for rep0 in range(100):
    seed = random.randint(0, 10000000)
#     seed = 3183013; n = 9
#     seed = 3259030; n = 10
#     seed = 7659167; n = 7
    seed = 2212498; n = 7
    print(f'{seed=}')
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    random.seed(seed + rep0)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge

    # steps 4.-6.
    graph = deepcopy(tree)
    eppdc = deepcopy(tree_eppdc)

    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        graph_before_part1 = deepcopy(graph)
        eppdc_before_part1 = deepcopy(eppdc)

        if verbose:
            print(f'before part 1; adding edge {(orig_v1, orig_v2)}')
            for path in eppdc:
                print(path)

        next_graphs = []
        next_eppdcs = []

        for orig_vs in [(orig_v1, orig_v2), (orig_v2, orig_v1)]:
            v1, v2 = orig_vs[0], orig_vs[1]

            reps = 50
            for rep in range(reps):
                graph = deepcopy(graph_before_part1)
                eppdc = deepcopy(eppdc_before_part1)

                success, _ = extend(graph, v1, v2, eppdc, 1,
                                    use_backtrack=False, verbose=verbose)
                if success == State.SUCCESS:
                    success, _ = extend(graph, v2, v1, eppdc, 2,
                                        use_backtrack=False, verbose=verbose)

                if success != State.FAIL:
                    assert success == State.SUCCESS
                    next_graphs.append(deepcopy(graph))
                    next_eppdcs.append(deepcopy(eppdc))
                if verbose:
                    time.sleep(0.02)

        if len(next_graphs) == 0:
            print(f'fail on adding ({v1}, {v2})')
            print(f'{edge_idx=}', 'vs', len(non_tree_edges))
            for path in eppdc_before_part1:
                print(path)
        elif edge_idx == 5:
            print("didn't fail")
            print(f'{edge_idx=}', 'vs', len(non_tree_edges))
            for path in eppdc_before_part1:
                print(path)
        assert (len(next_graphs) > 0)
        best_graph_idx = None
        best_val = None
        best_val2 = None

#         next_both = list(zip(next_graphs, next_eppdcs))
#         random.shuffle(next_both)
#         next_graphs, next_eppdcs = zip(*next_both)

        for graph_idx in range(len(next_graphs)):
            cur_val = next_eppdcs[graph_idx].calc_abs_sum_diffs_consec_path_lens()
            cur_val2 = next_eppdcs[graph_idx].calc_median_path_len()
#             cur_val3 = next_eppdcs[graph_idx].calc_repeated_triples()

            if best_val is None or (cur_val < best_val): # or ((cur_val == best_val) and (cur_val2 > best_val2)):
#             if random.random() > 0.25:
                best_graph_idx = graph_idx
                best_val = cur_val
                best_val2 = cur_val2
        graph = deepcopy(next_graphs[best_graph_idx])
        eppdc = deepcopy(next_eppdcs[best_graph_idx])


seed=2212498
didn't fail
edge_idx=5 vs 15
[0, 5, 3, 4]
[0, 3, 4, 2, 1]
[5, 4, 2, 3]
[4, 5, 6]
[5, 6, 3, 2]
[1, 0, 5, 3]
[6, 3, 0, 1, 2]
seed=2212498
didn't fail
edge_idx=5 vs 15
[0, 5, 4, 3, 2, 1]
[0, 3, 2, 4]
[2, 4, 3]
[4, 5, 6]
[5, 3, 6]
[3, 5, 0, 1]
[5, 6, 3, 0, 1, 2]
seed=2212498
didn't fail
edge_idx=5 vs 15
[0, 5, 3, 4]
[0, 3, 4, 2, 1]
[5, 4, 2, 3]
[4, 5, 6]
[5, 6, 3, 2]
[1, 0, 5, 3]
[6, 3, 0, 1, 2]
seed=2212498
fail on adding (3, 1)
edge_idx=5 vs 15
[0, 5, 6, 3, 4]
[0, 3, 4, 2, 1]
[5, 4, 2, 3]
[4, 5, 3, 6]
[6, 5, 3]
[5, 0, 1, 2]
[1, 0, 3, 2]


AssertionError: 

In [ ]:
# seed=3259030
# fail on adding (5, 2)
# edge_idx=18 vs 36
# [5, 7, 9, 2, 3, 0, 1, 8, 6, 4] 9
# [8, 3, 2, 0, 7, 9] 5
# [0, 3, 7, 4, 9, 1, 2] 6
# [6, 5, 4, 0, 2, 7] 5
# [3, 4, 7, 1, 9, 6] 5
# [7, 5, 6, 9, 4, 3, 8, 2] 7
# [3, 7, 1, 4, 6, 8, 5] 6
# [0, 4, 5, 8, 2, 1] 5
# [1, 8] 1
# [4, 1, 0, 7, 2, 9] 5

#  cycle: 5-4-9-8-1-0-2-7-6-3-5
# lengths: 9/5/5/1/5/6/


# [0, 1, 8, 2, 3, 7, 4, 6]
# [2, 3, 0, 7, 1, 9, 4]
# [1, 9, 2, 0, 4, 3, 7]
# [8, 5, 4, 7, 9]
# [5, 7, 2, 1, 4, 6, 9] 7
# [1, 2, 9, 6, 5, 7, 0] 7
# [6, 8, 5, 4, 3] 5
# [5, 6, 8, 3] 3
# [7, 1, 8, 2, 0, 4]
# [8, 3, 0, 1, 4, 9, 7, 2]


In [ ]:
# n = 10
# verbose = False
# seed = random.randint(0, 10000000)
# print(f'{seed=}')
# random.seed(seed)

# # 0. generate random order of edges
# edges = gen_edges(n)
# # 1. so that we actually start with building a tree
# tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)

# # 2. build EPPDC for this tree
# tree = Graph(n=n, edges=tree_edges)
# tree_eppdc = build_tree_eppdc(tree, n)
# if verbose:
#     for path in tree_eppdc:
#         print(path)

# # now we iteratively add an edge

# # steps 4.-6.
# graph = deepcopy(tree)
# eppdc = deepcopy(tree_eppdc)

# for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
#     next_graphs = []
#     next_eppdcs = []

#     for orig_vs in [(orig_v1, orig_v2), (orig_v2, orig_v1)]:
#         v1, v2 = orig_vs[0], orig_vs[1]

# #         backtrack_count = -1
# #         while True:
# #             backtrack_count += 1
#             # Hacky way to exit the unfortunate random choices
# #             upper_bound = 1  # graph.n ** 2
# #             if backtrack_count >= upper_bound:
# #                 break

#         graph_before_part1 = deepcopy(graph)
#         eppdc_before_part1 = deepcopy(eppdc)

#         reps = 4
#         for rep in range(reps):
#             use_backtrack = False  # FIXME
# #                 use_backtrack = (rep > reps / 2)

#             for idx, vp in enumerate([(v1, v2)]):  #, (v2, v1)]):
# #                     if idx == 1:
# #                         if verbose:
# #                             print('  stuck! try the other way around')

#                 graph = deepcopy(graph_before_part1)
#                 eppdc = deepcopy(eppdc_before_part1)

#                 if verbose:
#                     print(f'before part 1')
#                     for path in eppdc:
#                         print(path)
#                     print(f'adding edge ({vp[0]}, {vp[1]})')

#                 success, _ = extend(graph, vp[0], vp[1], eppdc, 1,
#                                     use_backtrack=use_backtrack, verbose=verbose)
#                 if success == State.SUCCESS:
#                     if verbose:
#                         print(f'  after part 1')
#                         for path in eppdc:
#                             print(path)
#                         print(f'  part2: adding edge ({vp[1]}, {vp[0]})')
#                     success, (new_v2, new_v1) = extend(graph, vp[1], vp[0], eppdc, 2,
#                                                        use_backtrack=use_backtrack, verbose=verbose)

#                 if verbose:
#                     time.sleep(0.01)
#                 if success != State.FAIL:
#                     break
#             if success != State.FAIL:
#                 assert success == State.SUCCESS
#                 next_graphs.append(deepcopy(graph))
#                 next_eepdcs.append(deepcopy(eppdc))
# #                     break
# #             assert success != State.FAIL
# #             eppdc.sanity_check()

# #             if verbose:
# #                 print('  after part 2')
# #                 for path in eppdc:
# #                     print(path)

# #             if success == State.SUCCESS:
# #                 break

# #             assert (success == State.BACKTRACK)
# #             graph_before_part1 = deepcopy(graph)
# #             eppdc_before_part1 = deepcopy(eppdc)
# #             if verbose:
# #                 print('SWITCH EDGE:', v1, v2, 'vs', new_v2, new_v1)
# #             v1, v2 = new_v2, new_v1
#     #         if backtrack_count > 0 or verbose:
# #         print(f'{backtrack_count=}', eppdc.calc_repeated_triples())

In [38]:
n = 5

dont_stop = True

# -1. set seed
verbose = False; seed = 0

# debugging examples
# n = 8; verbose = True; debug_seed = 307
# n = 8; verbose = True; debug_seed = 3
# n = 9; verbose = True; debug_seed = 723
# n = 7; verbose = True; debug_seed = 4818
# n = 7; verbose = True; debug_seed = 0
n = 15; verbose = False; debug_seed = random.randint(0, 10000000)

while seed < 1000 and dont_stop:
    if not verbose:
        seed += 1
    else:
        seed = debug_seed
        dont_stop = False
    seed = debug_seed
    dont_stop = False

    print(f'{seed=}')
    random.seed(seed)
    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge

    # steps 4.-6.
    graph = deepcopy(tree)
    eppdc = deepcopy(tree_eppdc)

    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
#         if edge_idx == len(non_tree_edges) - 1:  # FIXME
#             break

        v1, v2 = orig_v1, orig_v2
#         if random.random() < 0.5:  # FIXME
#             v1, v2 = v2, v1

        backtrack_count = -1
        while True:
            backtrack_count += 1
            # Hacky way to exit the unfortunate random choices
            upper_bound = graph.n ** 2
            if backtrack_count >= upper_bound:
                print(f'FAILED SEED {seed}')
            assert backtrack_count < upper_bound

            graph_before_part1 = deepcopy(graph)
            eppdc_before_part1 = deepcopy(eppdc)

            reps = 4
            for rep in range(reps):
#                 use_backtrack = True  # FIXME
                use_backtrack = (rep > reps / 2)

                for idx, vp in enumerate([[v1, v2], [v2, v1]]):
                    if idx == 1:
                        if verbose:
                            print('  stuck! try the other way around')

                    graph = deepcopy(graph_before_part1)
                    eppdc = deepcopy(eppdc_before_part1)

                    if verbose:
                        print(f'before part 1')
                        for path in eppdc:
                            print(path)
                        print(f'adding edge ({vp[0]}, {vp[1]})')

                    success, _ = extend(graph, vp[0], vp[1], eppdc, 1,
                                        use_backtrack=use_backtrack, verbose=verbose)
                    if success == State.SUCCESS:
                        if verbose:
                            print(f'  after part 1')
                            for path in eppdc:
                                print(path)
                            print(f'  part2: adding edge ({vp[1]}, {vp[0]})')
                        success, (new_v2, new_v1) = extend(graph, vp[1], vp[0], eppdc, 2,
                                                           use_backtrack=use_backtrack, verbose=verbose)

                    if verbose:
                        time.sleep(0.01)
                    if success != State.FAIL:
                        break
                if success != State.FAIL:
                    break
            assert success != State.FAIL
            eppdc.sanity_check()

            if verbose:
                print('  after part 2')
                for path in eppdc:
                    print(path)

            if success == State.SUCCESS:
                break

            assert (success == State.BACKTRACK)
            graph_before_part1 = deepcopy(graph)
            eppdc_before_part1 = deepcopy(eppdc)
            if verbose:
                print('SWITCH EDGE:', v1, v2, 'vs', new_v2, new_v1)
            v1, v2 = new_v2, new_v1
#         if backtrack_count > 0 or verbose:
        print(f'{backtrack_count=}', eppdc.calc_repeated_triples())

seed=2259736
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 0
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtrack_count=0 1
backtra

In [150]:
# TODO: 3./7. add visualizations to the whole process

# TODO: Yet to see this happen, or maybe not needed at all:
# 8. if the process fails - log this example;
# 9.                        and try to bruteforce 4.
# 10. if the process fails again - this is the end of this approach


In [ ]:
#         eligible_paths = []
#         for path in v1_paths:
#             if v2 not in path:
#                 eligible_paths.append(path)

#         if len(eligible_paths) > 0:
#             eligible_paths[0].append((v1, v2))
#             # TODO: maybe we would need to bruteforce/backtrack this choice
#         else:
#             assert False
#             # TODO: do the same thing, but recursively,
#             # and understand, that we don't lose eppdc
